# Algoritmo Minimax

Este notebook explica el algoritmo Minimax, un algoritmo de decisión clásico utilizado en teoría de juegos y inteligencia artificial para juegos de suma cero con información perfecta.

## ¿Qué es Minimax?

Minimax es un algoritmo recursivo de decisión que se utiliza para tomar decisiones óptimas en juegos de adversario (como ajedrez, tres en raya, damas, etc.). El algoritmo asume que ambos jugadores juegan de manera óptima.

## Conceptos Fundamentales

- **Jugador MAX**: Intenta maximizar su puntuación
- **Jugador MIN**: Intenta minimizar la puntuación de MAX
- **Árbol de juego**: Representación de todos los posibles movimientos
- **Función de evaluación**: Asigna un valor a cada estado del juego
- **Poda alfa-beta**: Optimización que reduce el número de nodos evaluados


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional


## Implementación Básica de Minimax

El algoritmo Minimax funciona de la siguiente manera:

1. Si el estado es terminal (fin del juego), retorna el valor de evaluación
2. Si es el turno de MAX, retorna el máximo valor de todos los posibles movimientos
3. Si es el turno de MIN, retorna el mínimo valor de todos los posibles movimientos


In [ ]:
def minimax(board, depth, is_maximizing, evaluate_func, get_moves_func, is_terminal_func):
    """
    Implementación básica del algoritmo Minimax
    
    Parámetros:
    - board: Estado actual del juego
    - depth: Profundidad actual en el árbol
    - is_maximizing: True si es el turno de MAX, False si es MIN
    - evaluate_func: Función que evalúa un estado del juego
    - get_moves_func: Función que retorna los posibles movimientos
    - is_terminal_func: Función que verifica si el juego terminó
    
    Retorna:
    - Valor de evaluación del estado
    """
    # Caso base: estado terminal o profundidad máxima alcanzada
    if is_terminal_func(board) or depth == 0:
        return evaluate_func(board)
    
    if is_maximizing:
        max_eval = float('-inf')
        for move in get_moves_func(board, True):
            eval_score = minimax(move, depth - 1, False, 
                                evaluate_func, get_moves_func, is_terminal_func)
            max_eval = max(max_eval, eval_score)
        return max_eval
    else:
        min_eval = float('inf')
        for move in get_moves_func(board, False):
            eval_score = minimax(move, depth - 1, True,
                                evaluate_func, get_moves_func, is_terminal_func)
            min_eval = min(min_eval, eval_score)
        return min_eval


## Ejemplo: Tres en Raya (Tic-Tac-Toe)

Vamos a implementar Minimax para el juego de tres en raya.


In [ ]:
class TicTacToe:
    """Clase para representar el juego de tres en raya"""
    
    def __init__(self, board=None):
        if board is None:
            self.board = [[' ' for _ in range(3)] for _ in range(3)]
        else:
            self.board = [row[:] for row in board]
    
    def print_board(self):
        """Imprime el tablero"""
        for i, row in enumerate(self.board):
            print(' | '.join(row))
            if i < 2:
                print('-' * 9)
    
    def is_winner(self, player):
        """Verifica si un jugador ha ganado"""
        # Filas
        for row in self.board:
            if all(cell == player for cell in row):
                return True
        # Columnas
        for col in range(3):
            if all(self.board[row][col] == player for row in range(3)):
                return True
        # Diagonales
        if all(self.board[i][i] == player for i in range(3)):
            return True
        if all(self.board[i][2-i] == player for i in range(3)):
            return True
        return False
    
    def is_full(self):
        """Verifica si el tablero está lleno"""
        return all(cell != ' ' for row in self.board for cell in row)
    
    def is_terminal(self):
        """Verifica si el juego terminó"""
        return self.is_winner('X') or self.is_winner('O') or self.is_full()
    
    def get_empty_cells(self):
        """Retorna las posiciones vacías"""
        return [(i, j) for i in range(3) for j in range(3) if self.board[i][j] == ' ']
    
    def make_move(self, row, col, player):
        """Hace un movimiento"""
        if self.board[row][col] == ' ':
            new_board = TicTacToe(self.board)
            new_board.board[row][col] = player
            return new_board
        return None


In [ ]:
def evaluate_tic_tac_toe(board):
    """Evalúa un estado del juego de tres en raya"""
    if board.is_winner('X'):  # MAX gana
        return 10
    elif board.is_winner('O'):  # MIN gana
        return -10
    else:
        return 0  # Empate o juego en curso

def get_moves_tic_tac_toe(board, is_maximizing):
    """Retorna todos los posibles movimientos"""
    moves = []
    player = 'X' if is_maximizing else 'O'
    for row, col in board.get_empty_cells():
        new_board = board.make_move(row, col, player)
        if new_board:
            moves.append(new_board)
    return moves

def is_terminal_tic_tac_toe(board):
    """Verifica si el juego terminó"""
    return board.is_terminal()


In [ ]:
def minimax_tic_tac_toe(board, depth, is_maximizing):
    """Minimax específico para tres en raya"""
    if is_terminal_tic_tac_toe(board) or depth == 0:
        return evaluate_tic_tac_toe(board)
    
    if is_maximizing:
        max_eval = float('-inf')
        for move in get_moves_tic_tac_toe(board, True):
            eval_score = minimax_tic_tac_toe(move, depth - 1, False)
            max_eval = max(max_eval, eval_score)
        return max_eval
    else:
        min_eval = float('inf')
        for move in get_moves_tic_tac_toe(board, False):
            eval_score = minimax_tic_tac_toe(move, depth - 1, True)
            min_eval = min(min_eval, eval_score)
        return min_eval

def find_best_move(board):
    """Encuentra el mejor movimiento para el jugador X (MAX)"""
    best_move = None
    best_value = float('-inf')
    
    for row, col in board.get_empty_cells():
        new_board = board.make_move(row, col, 'X')
        if new_board:
            move_value = minimax_tic_tac_toe(new_board, 9, False)
            if move_value > best_value:
                best_value = move_value
                best_move = (row, col)
    
    return best_move


In [ ]:
# Ejemplo de uso
game = TicTacToe()
print("Tablero inicial:")
game.print_board()

# Simular algunos movimientos
game.board[0][0] = 'X'
game.board[1][1] = 'O'
game.board[0][1] = 'X'

print("\nEstado después de algunos movimientos:")
game.print_board()

# Encontrar el mejor movimiento para X
best_move = find_best_move(game)
print(f"\nMejor movimiento para X: {best_move}")

# Aplicar el mejor movimiento
if best_move:
    game = game.make_move(best_move[0], best_move[1], 'X')
    print("\nTablero después del mejor movimiento:")
    game.print_board()


## Optimización: Poda Alfa-Beta

La poda alfa-beta es una optimización del algoritmo Minimax que reduce significativamente el número de nodos evaluados sin cambiar el resultado final. Funciona eliminando ramas del árbol que no pueden influir en la decisión final.


In [ ]:
def minimax_alpha_beta(board, depth, alpha, beta, is_maximizing, 
                       evaluate_func, get_moves_func, is_terminal_func):
    """
    Implementación de Minimax con poda alfa-beta
    
    Parámetros:
    - alpha: Mejor valor que MAX puede garantizar
    - beta: Mejor valor que MIN puede garantizar
    """
    if is_terminal_func(board) or depth == 0:
        return evaluate_func(board)
    
    if is_maximizing:
        max_eval = float('-inf')
        for move in get_moves_func(board, True):
            eval_score = minimax_alpha_beta(move, depth - 1, alpha, beta, False,
                                           evaluate_func, get_moves_func, is_terminal_func)
            max_eval = max(max_eval, eval_score)
            alpha = max(alpha, eval_score)
            # Poda beta
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in get_moves_func(board, False):
            eval_score = minimax_alpha_beta(move, depth - 1, alpha, beta, True,
                                           evaluate_func, get_moves_func, is_terminal_func)
            min_eval = min(min_eval, eval_score)
            beta = min(beta, eval_score)
            # Poda alfa
            if beta <= alpha:
                break
        return min_eval

def minimax_alpha_beta_tic_tac_toe(board, depth, alpha, beta, is_maximizing):
    """Minimax con poda alfa-beta para tres en raya"""
    if is_terminal_tic_tac_toe(board) or depth == 0:
        return evaluate_tic_tac_toe(board)
    
    if is_maximizing:
        max_eval = float('-inf')
        for move in get_moves_tic_tac_toe(board, True):
            eval_score = minimax_alpha_beta_tic_tac_toe(move, depth - 1, alpha, beta, False)
            max_eval = max(max_eval, eval_score)
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in get_moves_tic_tac_toe(board, False):
            eval_score = minimax_alpha_beta_tic_tac_toe(move, depth - 1, alpha, beta, True)
            min_eval = min(min_eval, eval_score)
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval


In [ ]:
# Contador de nodos evaluados
nodes_evaluated_minimax = 0
nodes_evaluated_ab = 0

def minimax_with_counter(board, depth, is_maximizing):
    """Minimax con contador de nodos"""
    global nodes_evaluated_minimax
    nodes_evaluated_minimax += 1
    
    if is_terminal_tic_tac_toe(board) or depth == 0:
        return evaluate_tic_tac_toe(board)
    
    if is_maximizing:
        max_eval = float('-inf')
        for move in get_moves_tic_tac_toe(board, True):
            eval_score = minimax_with_counter(move, depth - 1, False)
            max_eval = max(max_eval, eval_score)
        return max_eval
    else:
        min_eval = float('inf')
        for move in get_moves_tic_tac_toe(board, False):
            eval_score = minimax_with_counter(move, depth - 1, True)
            min_eval = min(min_eval, eval_score)
        return min_eval

def minimax_ab_with_counter(board, depth, alpha, beta, is_maximizing):
    """Minimax con poda alfa-beta y contador"""
    global nodes_evaluated_ab
    nodes_evaluated_ab += 1
    
    if is_terminal_tic_tac_toe(board) or depth == 0:
        return evaluate_tic_tac_toe(board)
    
    if is_maximizing:
        max_eval = float('-inf')
        for move in get_moves_tic_tac_toe(board, True):
            eval_score = minimax_ab_with_counter(move, depth - 1, alpha, beta, False)
            max_eval = max(max_eval, eval_score)
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in get_moves_tic_tac_toe(board, False):
            eval_score = minimax_ab_with_counter(move, depth - 1, alpha, beta, True)
            min_eval = min(min_eval, eval_score)
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval

# Probar con un tablero
test_game = TicTacToe()
test_game.board[0][0] = 'X'
test_game.board[1][1] = 'O'

# Resetear contadores
nodes_evaluated_minimax = 0
nodes_evaluated_ab = 0

# Evaluar con Minimax normal
_ = minimax_with_counter(test_game, 7, True)
minimax_nodes = nodes_evaluated_minimax

# Evaluar con Minimax + poda alfa-beta
_ = minimax_ab_with_counter(test_game, 7, float('-inf'), float('inf'), True)
ab_nodes = nodes_evaluated_ab

print(f"Nodos evaluados con Minimax: {minimax_nodes}")
print(f"Nodos evaluados con Minimax + Poda Alfa-Beta: {ab_nodes}")
print(f"Reducción: {((minimax_nodes - ab_nodes) / minimax_nodes * 100):.1f}%")
